### statically-typed language : java,C, C++
 컴파일시에 에러가 검출된다. 타입구분이 명확한것들을 정적타입 랭귀지 라함.
 
### dynamic-typed language : python, javascript, Ruby... 
 인터프리터 언어라고도한다. 타입에 딱히 신경쓰지않아도 개발이가능하며 개발속도 기본적으로 빠르다.
 
IDE에서 statical language 는 코드 suggestion이 잘 적용되지만, python같은 dynamic language 는 지원이 상대적으로 부족하다.

In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
from attention import Batcher, construct_feed_dict, extract_results, get_evals,AttentionModel,get_initial_state

In [2]:
data= 'data_samples/'
data_path='data_samples/mapping.map'
# pattern = 'preprocess.part*'
pattern = 'output.txt.part*'
hidden_size = 50
seq_length = 10
batch_size = 16
epochs=5
with open(data_path, "rb") as f:
    word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
num_samples = 3
attention_num=5
max_attention=3
lambda_type='state'
keep_prob=0.9
num_layer=1
max_grad_norm=3
learning_rate = 0.01
# files = [y for x in os.walk(data) for y in iglob(os.path.join(x[0], pattern))]

### 전처리를 다음과같이 한다.

#### 변수명:번호 인데 아래와 같이 나온다.

In [3]:
# current_file = deque(files).popleft()
current_file = './data_samples/output.txt.part0'
# current_file = files[0]
with open(current_file, 'rb') as f:
    current_data = pickle.load(f)
    print(current_file)
    print(current_data)

./data_samples/output.txt.part0
[<pyreader.Container object at 0x0000013A772ACA20>, <pyreader.Container object at 0x0000013A7D278898>, <pyreader.Container object at 0x0000013A7D2787B8>, <pyreader.Container object at 0x0000013A7D278A90>, <pyreader.Container object at 0x0000013A7D278BA8>, <pyreader.Container object at 0x0000013A7D278C88>, <pyreader.Container object at 0x0000013A7D278DA0>, <pyreader.Container object at 0x0000013A7D278E80>, <pyreader.Container object at 0x0000013A7D278F60>, <pyreader.Container object at 0x0000013A7D3C3358>, <pyreader.Container object at 0x0000013A7D3C3470>, <pyreader.Container object at 0x0000013A7D3C3588>, <pyreader.Container object at 0x0000013A7D3C36A0>, <pyreader.Container object at 0x0000013A7D3C37B8>, <pyreader.Container object at 0x0000013A7D3C38D0>, <pyreader.Container object at 0x0000013A7D3C39E8>, <pyreader.Container object at 0x0000013A7D3C3B00>, <pyreader.Container object at 0x0000013A7D3C3C18>, <pyreader.Container object at 0x0000013A7D3C3D30>

In [4]:
current_data[1].inputs[0]

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [5]:
current_data[1].actual_lengths

[0]

### 예를들어 파일한개를 돌려보면..

In [6]:
inv_map = {v: k for k, v in word_to_id.items()} # 아이디값:변수명 으로 reverse시킴.
inv_map[38]
inv_map[10]

'§<indent>§'

In [7]:
for i in range(len(current_data)):
    inputs_raw = [inv_map[k] for k in current_data[i].inputs[0]]
#     inputs_raw = [inv_map[k] for k in current_data[i].identifier_usage[0]]
#     inputs_raw = [k for k in current_data[i].masks]
    inputs_raw= ' '.join(inputs_raw)
    print(inputs_raw)
    targets_raw = [inv_map[k] for k in current_data[i].targets[0]]
    targets_raws = ' '.join(targets_raw)
print(word_to_id['class'],inv_map[56])
# print(inputs_raw)
# print('--------------------------------------------------------------------------')
# print(targets_raws)

§OOV§ 
 var961 = §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§
from §OOV§ import §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§
var828 = §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§
var1510 = §OOV§ 
 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§ §PAD§

 §P

from django . contrib . gis . sitemaps import §OOV§
from django . conf . urls import url 
 from
from django . conf . urls import url 
 from
from django . conf . urls import url 
 from
from django . conf . urls import url 
 from
from django . conf . urls import url 
 from
from django . conf . urls import url 
 from
from django . db import migrations , models 
 

from django . db import migrations 
 
 
 class
from django . db import models 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migrations 
 
 
 class
from django . db import migra

 from
import json 
 from datetime import timedelta 
 from django
from __future__ import unicode_literals 
 from django . core .
from django . conf import settings 
 from django .
§OOV§ 
 import random 
 import tempfile 
 from django
from django . core import management 
 from django .
§OOV§ 
 from django . db import models 
 

import uuid 
 from datetime import datetime 
 from time
from unittest import mock 
 from urllib . parse import
from django . test import SimpleTestCase 
 from django .
from django . http import HttpResponse , StreamingHttpResponse 
 from
§OOV§ 
 from __future__ import absolute_import 
 from __future__ import
import re 
 import sys 
 import os 
 from
from __future__ import unicode_literals 
 import six 
 
 

from django . contrib . sites . models import Site
import re 
 from django . utils . cache import
from django . http import HttpResponse 
 from django .
import json 
 from django . contrib . contenttypes .
from django . forms import CharField , §OOV§ , EmailFi

In [8]:
current_data[40].inputs[0].shape

(10,)

### 위와같이 normalise + indent와 패딩, out of vocabulary의 값들이  나온다.

### 실제로 집어넣을 값들은 숫자로 feeding한다.

In [9]:
current_data[i].inputs[0]

array([ 20,  37,   2,  20, 325,   2,  20,  59,   2,  20], dtype=int64)

### 위와같이 넣게되어있다.

In [10]:
with tf.Graph().as_default(), tf.Session() as session:
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, 1], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=True,
                                 attention_num= 1,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                 max_attention=max_attention)
    
    session.run(tf.global_variables_initializer())
    loss = a.loss
    train = a.train_op 
    evals = [loss, train]
    evals = get_evals(evals=evals, model=a)
    state, att_states, att_ids, att_counts = get_initial_state(a, session)
    lr = learning_rate
    a.assign_lr(session, lr)
    
    batcher = Batcher([current_file], batch_size, seq_length)
    for epoch in range(epochs):
        
        for batch in batcher:
            
            total_loss = 0
            total_length = 0
                
            for feed_data in batcher.sequence_iterator(batch):

                feed_dict, identifiers_usage = construct_feed_dict(a, feed_data, state, att_states, att_ids, att_counts)
                results = session.run(evals, feed_dict= feed_dict)

                results, state, att_states, att_ids, alpha_states, att_counts, lambda_state= extract_results(results,evals, 2,a)
                
                total_loss += sum(results[0])
                total_length += sum(feed_dict[a._actual_lengths])
        print('perplexity ', np.exp(total_loss/total_length), 'epoch ', epoch)
        saver = tf.train.Saver(tf.trainable_variables())
        now = datetime.datetime.now().strftime("%Y-%m-%d--%H-%M--%f")
        model_path = './model'
        out_path = os.path.join(model_path, now + "/")
        saver.save(session, os.path.join(out_path, "model.tf"))

Constructing Attention Cell
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  2.27084572619e+13 epoch  0
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  2.1719894993e+13 epoch  1
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  1.78899909783e+13 epoch  2
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  1.435993796e+13 epoch  3
Loaded data partition ./data_samples/output.txt.part0 with 2515 examples
perplexity  1.18692786211e+13 epoch  4


In [11]:
model_path = './model/model.tf'
variables =  tf.trainable_variables()
saver = tf.train.Saver(variables)
saver.restore(session, path)

ValueError: No variables to save

In [16]:
def map_token(map, token):
    mask = 0
    if token.startswith("(*) "):
        mask = 1
        token = token.replace("(*) ", "")

    if token in map:
        return map[token], mask

    # Not in map, is it an identifier?
    if "|" in token:
        spl = token.split("|")
        if spl[1] in map:
            return map[spl[1]]
        elif spl[0] in map:
            return map[spl[0]]

    elif any([token.startswith(prefix) for prefix in astwalker.possible_types()]):
        return pyreader.oov_id

    raise KeyError(token)

In [25]:
test_cases =["import", "os", "\n", "\n", "class", "Class253", ":", "\n", "\n", "§<indent>§",
     "def", "__init__", "(", "self", ",", "arg651", ")", ":", "\n", "§<indent>§",
     "self", ".", "attribute943", "=", "arg651", "\n", "\n", "§<dedent>§",
     "def", "function1690", "(", "self", ",", "arg2004", ")", ":", "\n", "§<indent>§",
     "var4040", "=", "os", ".", "path", ".", "join", "(", "self", ".", "attribute943", ",", "arg2004", ")", "\n",
     "print", "(", "§OOV§", "%", "(", "str", "("
     ]
# state, att_states, att_ids, att_counts = get_initial_state(a, session)
for i, token in enumerate(test_cases[:-3]):
    print(map_token(word_to_id, token))

# for i, token in enumerate(testcase[:-depth]):
#     results, state, att_states, att_counts = run_network(map_token(word_to_id, token), state, att_states,
#                                                                      att_ids, att_counts)
# session.run(evals, feed_dict=)

(20, 0)
(59, 0)
(2, 0)
(2, 0)
(23, 0)
(2417, 0)
(8, 0)
(2, 0)
(2, 0)
(10, 0)
(15, 0)
(135, 0)
(3, 0)
(9, 0)
(6, 0)
(10540, 0)
(4, 0)
(8, 0)
(2, 0)
(10, 0)
(9, 0)
(5, 0)


NameError: name 'astwalker' is not defined